In [14]:
pip install gender-guesser

Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
import numpy as np
import re
import gender_guesser.detector as gender

In [2]:
df = pd.read_csv("final_spotify_wrapped_by_genres.csv", index_col=False)

In [3]:
# Dataset de Florish para renombrar los paises, ex: ES a Spain
regions = pd.read_csv('regions.csv', index_col=False)
regions2 = regions[['Country name', 'WB_A2']]
regions2 = regions2.rename(columns={'WB_A2':'country'}) 

# Crear dataset para el mapa

In [4]:
mapa = df[['name','artists','daily_rank','country']]
canciones_por_pais = mapa.groupby('country')['name'].nunique().reset_index()
canciones_por_pais.rename(columns={'name': 'Total Unique Songs in 46 days'}, inplace=True)

mapa = pd.merge(mapa, canciones_por_pais, on='country', how='left')
mapa = pd.merge(mapa, regions2, on = 'country', how='left')
mapa.drop('country', axis=1, inplace=True)

In [5]:
mapa2 = mapa[['Country name', 'Total Unique Songs in 46 days']].drop_duplicates(subset='Country name', keep='last').reset_index(drop=True)
mapa3 = mapa2.reset_index()

In [148]:
mapa3.to_csv('mapa.csv', index=False)

# Crear dataset con características medias

In [8]:
medias = df[['daily_rank', 'snapshot_date', 'valence','duration_ms', 'danceability',
             'energy','loudness', 'speechiness', 'acousticness', 'instrumentalness']]
filtered_medias = medias[medias['daily_rank'].isin([1, 2, 3, 4, 5])].drop('daily_rank', axis=1)
nuevo_dataset = filtered_medias.groupby('snapshot_date').mean().reset_index()

In [9]:
columnas_a_convertir = ['energy', 'speechiness', 'instrumentalness', 'valence', 'danceability', 'acousticness']
nuevo_dataset[columnas_a_convertir] = nuevo_dataset[columnas_a_convertir] * 100

In [10]:
# Determinar los valores mínimo y máximo de 'loudness'
min_loudness = nuevo_dataset['loudness'].min()
max_loudness = nuevo_dataset['loudness'].max()

# Definir los límites para las categorías
categorias = {
    'Muy Bajo': (min_loudness, -20),
    'Bajo': (-20, -10),
    'Moderado': (-10, -5),
    'Alto': (-5, 0),
    'Muy Alto': (0, max_loudness)
}

# Función para asignar una categoría basada en el valor de 'loudness'
def asignar_categoria(valor):
    for categoria, (min_valor, max_valor) in categorias.items():
        if min_valor <= valor <= max_valor:
            return categoria
    return 'Desconocido'

# Crear una nueva columna 'loudness_category' con las categorías asignadas
nuevo_dataset['loudness_category'] = nuevo_dataset['loudness'].apply(asignar_categoria)

# Mostrar el DataFrame con la nueva columna de categorías
nuevo_dataset.head()

,snapshot_date,valence,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,loudness_category
0,2023-10-18,53.272603,194078.967123,72.536438,64.661918,-6.579953,12.415973,31.223293,1.464216,Moderado
1,2023-10-19,53.272603,194078.967123,72.536438,64.661918,-6.579953,12.415973,31.223293,1.464216,Moderado
2,2023-10-20,54.313151,192131.975342,72.975616,64.419178,-6.607803,12.291863,30.964690,1.699442,Moderado
3,2023-10-21,55.124384,190706.186301,73.062740,64.882740,-6.492162,12.150521,31.082402,1.632241,Moderado
4,2023-10-22,56.136438,189093.106849,73.372603,65.658904,-6.384849,12.114055,29.479186,1.068133,Moderado


In [11]:
nuevo_dataset.describe()

,valence,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness
count,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000
mean,56.171872,188520.212812,71.696997,65.698047,-6.381323,10.607129,29.667719,1.543141
std,1.414849,3054.605930,1.049243,0.697863,0.135403,0.806387,1.076274,0.245316
min,52.574603,182663.459701,68.475068,64.419178,-6.607803,9.009863,26.102769,1.068133
25%,55.625315,185902.865753,71.534384,65.149315,-6.476025,10.105425,29.025229,1.354691
50%,56.409068,188358.901370,71.841644,65.700548,-6.384866,10.549973,29.692932,1.504533
75%,56.996046,190497.967123,72.315479,66.184370,-6.257134,10.993499,30.310771,1.686121
max,59.227808,194793.090411,73.372603,67.375068,-6.026937,12.415973,31.558211,2.230880


In [12]:
nuevo_dataset.to_csv('medias.csv', index=False)

# Crear dataset para la brecha de género

In [13]:
brecha = df[['artists', 'country']]

In [17]:
# Crear un objeto detector
detector = gender.Detector()

# Suponiendo que 'brecha' es tu DataFrame y 'artists' es la columna de nombres de artistas
# Obtener el género basado en el nombre
brecha['predicted_gender'] = brecha['artists'].astype(str).apply(lambda x: detector.get_gender(x.split()[-1]))

# Contar géneros por país
genero_por_pais = brecha.groupby(['country', 'predicted_gender']).size().unstack().fillna(0)

C:\Users\isa31\AppData\Local\Temp/ipykernel_21772/3240492033.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brecha['predicted_gender'] = brecha['artists'].astype(str).apply(lambda x: detector.get_gender(x.split()[-1]))


In [18]:
brecha2 = brecha.drop_duplicates().reset_index(drop=True)

In [19]:
brecha2['gender_count'] = brecha2.groupby(['country', 'predicted_gender'])['country'].transform('count')

In [20]:
brecha3 = brecha2.drop('artists', axis=1).drop_duplicates().reset_index(drop=True)

In [21]:
pivot_table = pd.pivot_table(brecha3, values='gender_count', index='country', columns='predicted_gender', fill_value=0)
pivot_table.reset_index(inplace=True)

In [33]:
pivot_table.to_csv('brecha_final.csv', index=False)

# Crear dataset géneros musicales

In [25]:
genres = df[['genre', 'snapshot_date']]
genres['snapshot_date'] = pd.to_datetime(genres['snapshot_date'])

octubre = genres[genres['snapshot_date'].dt.month == 10]
noviembre = genres[genres['snapshot_date'].dt.month == 11]
diciembre = genres[genres['snapshot_date'].dt.month == 12]

conteos_octubre = octubre['genre'].value_counts(normalize=True) * 100
conteos_noviembre = noviembre['genre'].value_counts(normalize=True) * 100
conteos_diciembre = diciembre['genre'].value_counts(normalize=True) * 100

todos_los_generos = list(set(conteos_octubre.index) | set(conteos_noviembre.index) | set(conteos_diciembre.index))
resultados = pd.DataFrame({
    'Género': todos_los_generos,
    'Octubre': conteos_octubre.reindex(todos_los_generos, fill_value=0).values,
    'Noviembre': conteos_noviembre.reindex(todos_los_generos, fill_value=0).values,
    'Diciembre': conteos_diciembre.reindex(todos_los_generos, fill_value=0).values
})

print(resultados)

         Género   Octubre  Noviembre  Diciembre
0     breakbeat  0.209377   0.205450   0.219178
1   world-music  0.084142   0.037605   0.036530
2        pagode  0.000000   0.003669   0.000000
3         sleep  0.043049   0.030267   0.036530
4        brazil  0.093926   0.120152   0.063927
..          ...       ...        ...        ...
77       techno  0.720100   0.938282   1.022831
78      spanish  0.207420   0.198112   0.210046
79        chill  0.849249   0.797036   0.557078
80       j-idol  0.334612   0.394390   0.401826
81     children  0.311130   0.422823   0.356164

[82 rows x 4 columns]


C:\Users\isa31\AppData\Local\Temp/ipykernel_21772/1002795380.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genres['snapshot_date'] = pd.to_datetime(genres['snapshot_date'])


In [174]:
resultados.to_csv('genres_per_month.csv', index = False)

# Crear dataset is_explicit

In [162]:
explicito = df[['is_explicit']]
porcentaje_explicito = explicito['is_explicit'].value_counts(normalize=True) * 100
porcentaje_explicito

False    62.842597
True     37.157403
Name: is_explicit, dtype: float64

# Crear dataset top 5 en todo el mundo

In [35]:
top5global = df[['artists']]

top5global = top5global.assign(artists=top5global['artists'].str.split(',')).explode('artists')
top5global.reset_index(drop=True, inplace=True)

top5global['artists'] = top5global['artists'].str.strip()
top5global['artist_count'] = top5global['artists'].map(top5global['artists'].value_counts())
top5global = top5global.sort_values(by='artist_count', ascending=False).drop_duplicates()

In [49]:
top5global.head()

,artists,artist_count
153259,Bad Bunny,9823.0
178416,Taylor Swift,5528.0
217505,Feid,4518.0
47647,Jung Kook,4234.0
92552,Peso Pluma,3037.0


In [50]:
top5global.to_csv('top5global_total.csv', index=False)

# Crear dataset Top 5 en una semana

In [26]:
top5_date = df[['artists', 'daily_rank', 'snapshot_date', 'country']]
top5_date_filtered = top5_date[(top5_date['daily_rank'] >= 1) & (top5_date['daily_rank'] <= 5)]
top5_date_filtered = top5_date_filtered[(top5_date_filtered['snapshot_date'] >= '2023-11-26') & (top5_date_filtered['snapshot_date'] <= '2023-12-03')]

In [27]:
pivot_table = pd.pivot_table(top5_date_filtered, values='daily_rank', index=['artists', 'country'], columns='snapshot_date', fill_value=None)
pivot_table.reset_index(inplace=True)

In [81]:
pivot_table = pd.merge(pivot_table, regions2, on = 'country', how='left')
pivot_table.drop('country', axis=1, inplace=True)

In [83]:
pivot_table

,artists,2023-11-26,2023-11-27,2023-11-28,2023-11-29,2023-11-30,2023-12-01,2023-12-02,2023-12-03,Country name
0,ABREU,3.0,4.0,4.0,4.0,5.0,NaN,NaN,NaN,Finland
1,AIGEL,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Belarus
2,AIGEL,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,Kazakhstan
3,AIGEL,5.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,Ukraine
4,AUR,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,India
...,...,...,...,...,...,...,...,...,...,...
537,יגל אושרי,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Israel
538,ששון איפרם שאולוב,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,Israel
539,卢卢快闭嘴,4.0,4.0,NaN,5.0,5.0,5.0,NaN,NaN,NaN
540,承桓,3.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0,NaN


In [84]:
pivot_table.to_csv('ranking.csv', index = False)

# Canibalizacion de la Taylor al bad bunny

In [28]:
canibal = df[['artists', 'daily_rank', 'snapshot_date', 'country']]
canibal_filtrado = canibal[canibal['artists'].isin(['Bad Bunny', 'Taylor Swift'])]

In [29]:
artistas_interesantes = ['Bad Bunny', 'Taylor Swift']

conteo_artistas_por_pais_fecha = canibal_filtrado.groupby(['country', 'snapshot_date'])['artists'].nunique()
paises_con_ambos_artistas = conteo_artistas_por_pais_fecha[conteo_artistas_por_pais_fecha == len(artistas_interesantes)].index

# Filtrar el DataFrame original con los países que cumplen la condición
canibal_final = canibal_filtrado.set_index(['country', 'snapshot_date']).loc[paises_con_ambos_artistas].reset_index()
canibal_final = canibal_final.sort_values(by=['country', 'snapshot_date', 'daily_rank'], ascending=True).drop_duplicates(subset=['country', 'snapshot_date', 'artists'], keep='first')

In [30]:
pivot_table = pd.pivot_table(canibal_final, values='daily_rank', index=['artists', 'country'], columns='snapshot_date', fill_value=None)
pivot_table.reset_index(inplace=True)

pivot_table = pd.merge(pivot_table, regions2, on = 'country', how='left')
pivot_table.drop('country', axis=1, inplace=True)

In [129]:
pivot_table.to_csv('canibalización.csv', index=False)

# Conteo mas Top 10 con canciones unicas

In [32]:
import pandas as pd

dff = df[['artists', 'name', 'album_name', 'snapshot_date', 'country', 'daily_rank']]

# Filtrar los datos para las posiciones del 1 al 10 y canciones distintas
df_top10_unique = dff[dff['daily_rank'].between(1, 10)]

# Agrupar por artistas, países y fecha, y contar las ocurrencias
result = df_top10_unique.groupby(['artists', 'country', 'snapshot_date']).agg({'name': 'nunique', 'daily_rank': 'count'}).reset_index()
result = result.rename(columns={'name': 'count_unique_songs', 'daily_rank': 'count_rankings'})

# Filtrar los resultados donde count sea mayor a 1 (ocurrió en el mismo día con canciones distintas)
result = result[(result['count_rankings'] > 1) & (result['count_unique_songs'] == 10)]

result = result.sort_values(by='count_rankings', ascending=False)

# Crear la nueva columna "count_total"
result['count_total'] = result.groupby('artists')['count_rankings'].transform('sum')

# Mostrar el DataFrame final
result = result.sort_values(by='count_total', ascending=False)
result.drop(['count_rankings', 'count_unique_songs'], axis=1, inplace=True)
result.drop_duplicates(inplace=True)
result.to_csv('top10_monopolizado.csv', index=False)
